In [1]:
import numpy as np
import cv2
import scipy
import matplotlib.pyplot as plt

In [2]:
def createGaussianMask(image_size, sigma_x, sigma_y):
    cols, rows = image_size
    centre_x, centre_y = rows / 2, cols / 2
    x = np.linspace(0, rows, rows)
    y = np.linspace(0, cols, cols)
    X, Y = np.meshgrid(x,y)
    mask = np.exp(-(np.square((X - centre_x)/sigma_x) + np.square((Y - centre_y)/sigma_y)))
    return mask

In [3]:
def findContour(image):
    ret,thresh = cv2.threshold(np.uint8(image), 200 ,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    chosen_contours = []

    # index1 = np.where(hierarchy[0, :, 3] == -1) 
    # hierarchy_no_parent = hierarchy[:, index1, :]
    # index2 = np.where(hierarchy_no_parent[0, :, 2] != -1)
    # hierarchy_no_parent_yes_child = hierarchy_no_parent[:, index2, :]
    

    previous_area = cv2.contourArea(contours[0])
    for j in range(len(contours)):
        if hierarchy[0, j, 3] == -1:#no parent
            if hierarchy[0, j, 2] !=-1: #child
                #print("no parent, child present")
                area = cv2.contourArea(contours[j])
                if True:#np.abs(area - previous_area) < 10000:
                    chosen_contours.append(contours[j])
                    previous_area = area

    return chosen_contours

In [4]:
def findCornersFromContour(contour):
    x_min, y_min = np.min(contour, axis = 0)
    x_max, y_max = np.max(contour, axis = 0)

    index1 = np.where(contour[:,0] == x_min) #topmost point
    index1 = index1[0]
    index1 = index1[0]
    x1 = contour[index1,0]
    y1 = contour[index1,1]

    index2 = np.where(contour[:,1] == y_max) #right most point
    index2 = index2[0]
    index2 = index2[0]
    x2 = contour[index2, 0]
    y2 = contour[index2, 1]

    index3 = np.where(contour[:,0] == x_max) #bottommost point
    index3 = index3[0]
    index3 = index3[0]
    x3 = contour[index3, 0]
    y3 = contour[index3, 1]

    index4 = np.where(contour[:,1] == y_min) #leftmost point
    index4 = index4[0]
    index4 = index4[0]
    x4 = contour[index4, 0]
    y4 = contour[index4, 1]

    corner = np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
    return corner

In [5]:
BasePath = '/home/sakshi/courses/ENPM673/sakshi_p1/'
video_file = BasePath + "Data/multipleTags.mp4"
cap = cv2.VideoCapture(video_file)

while(True):
    ret, frame = cap.read()
    if not ret:
        print("Stream ended..")
        break
    
    image_rgb = frame
    image_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    image_gray = cv2.GaussianBlur(image_gray, (21, 21), 0)

    chosen_contours = findContour(image_gray)
    out_mask = np.zeros_like(image_gray)
    corners = []
    for chosen_contour in chosen_contours:
        # corner = findCornersFromContour(chosen_contour.reshape(-1,2))
        # corners.append(corner)
        corner = cv2.approxPolyDP(chosen_contour, 0.009 * cv2.arcLength(chosen_contour, True), True)
        corners.append(corner.reshape(-1,2))
        cv2.drawContours(image_rgb, chosen_contour, -1, (0, 0 ,255), 3, lineType = cv2.LINE_8)
        cv2.drawContours(out_mask, [chosen_contour], -1, 1, cv2.FILLED)  

    for c in corners:
        for r in range(c.shape[0]):
            x,y = c[r,:]
            cv2.circle(image_rgb,(x,y),20,(255, 0, 255),-1)   

        
    cv2.imshow('frame',image_rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Stream ended..
